In [7]:
import numpy as np
import pandas as pd
import re
import argparse
import sys
import os
import warnings

from pyminc.volumes.factory import *
from glob                   import glob

from sklearn.impute         import KNNImputer
from sklearn.preprocessing  import FunctionTransformer
from sklearn.pipeline       import Pipeline

In [21]:
datadir = 'data/expression/'
outdir = 'data/'
dataset = 'coronal'
maskFlag = 'coronal'
groupExperiments = True
impute = False

if (dataset == 'sagittal') and (maskFlag == 'coronal'):
        warnings.warn("Running with sagittal dataset and coronal mask is not ideal. Proceed with caution.")

#If dataset is sagittal, use only those genes that are also in the coronal set
if dataset == "sagittal":

    #Paths to sagittal and coronal data set directories
    pathGeneDir_Sagittal = os.path.join(datadir, dataset, '')
    pathGeneDir_Coronal = os.path.join(datadir, 'coronal', '')

    #Build paths to all files in the directories
    pathGeneFiles_Sagittal = np.array(glob(pathGeneDir_Sagittal + "*.mnc"))
    pathGeneFiles_Coronal = np.array(glob(pathGeneDir_Coronal + "*.mnc"))

    #Extract gene names for coronal and sagittal data sets
    genes_Sagittal = np.array([re.sub(r"_[0-9]+.mnc", "", file) for file in 
                                [basename(path) for path in pathGeneFiles_Sagittal]])
    genes_Coronal = np.array([re.sub(r"_[0-9]+.mnc", "", file) for file in 
                                [basename(path) for path in pathGeneFiles_Coronal]])

    #Identify genes from sagittal data in coronal data
    isInCoronal = np.isin(genes_Sagittal, genes_Coronal)

    #Extract subset of sagittal gene files
    pathGeneFiles = pathGeneFiles_Sagittal[isInCoronal]

else:
    pathGeneDir = os.path.join(datadir, dataset, '')
    pathGeneFiles = np.array(glob(pathGeneDir+"*.mnc"))
    
#Load image mask and convert to numpy array
if maskFlag == "sagittal":
    maskfile = "data/imaging/sagittal_200um_coverage_bin0.8.mnc"
    maskVol = volumeFromFile(maskfile)        
else: 
    maskfile = "data/imaging/coronal_200um_coverage_bin0.8.mnc"
    maskVol = volumeFromFile(maskfile)

maskArray = np.array(maskVol.data.flatten())
maskVol.closeVolume()

In [27]:
def importImage(infile, mask):
    
    """ """
    
    #Read ISH data to numpy array
    exprVol = volumeFromFile(infile)
    exprArray = np.array(exprVol.data.flatten())
    exprVol.closeVolume()
    
    #Apply mask and convert -1 to NaN
    exprArrayMasked = exprArray[mask == 1]
    exprArrayMasked[exprArrayMasked == -1] = np.nan
    exprArrayMasked[exprArrayMasked == 0] = np.nan
    
    return exprArrayMasked

In [56]:
from functools import partial
from tqdm import tqdm
import multiprocessing as mp

In [79]:
files = pathGeneFiles
mask = maskArray
threshold = 0.2
group_experiments = True
parallel = True
nproc = 2

importImage_partial = partial(importImage, mask = mask)

if parallel:

    if nproc is None:
        nproc = mp.cpu_count()

    pool = mp.Pool(nproc)

    results = []
    for result in tqdm(pool.imap(importImage_partial, files), total = len(files)):
        results.append(result)

    pool.close()
    pool.join()

else:

    results = list(map(importImage_partial, tqdm(files)))
    
dfExpression = pd.DataFrame(np.asarray(results), index = [os.path.basename(file) for file in files])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4345/4345 [01:17<00:00, 56.32it/s]


In [81]:
dfExpression.shape

(4345, 61315)

In [50]:
def buildExpressionMatrix(files, mask, log_transform = True, group_experiments = True, missing_threshold = 0.2, parallel = True, nproc = None):
    
    """ """
    
    importImage_partial = partial(importImage, mask = mask)

    if parallel:

        if nproc is None:
            nproc = mp.cpu_count()

        pool = mp.Pool(nproc)

        arrays = []
        for array in tqdm(pool.imap(importImage_partial, files), total = len(files)):
            arrays.append(array)

        pool.close()
        pool.join()

    else:

        arrays = list(map(importImage_partial, tqdm(files)))
    
    dfExpression = pd.DataFrame(np.asarray(results), index = [os.path.basename(file) for file in files])

    #Transform to log2
    if log_transform:
        print("Applying log2 transform...")
        dfExpression = np.log2(dfExpression)
    
    dfExpression.index = dfExpression.index.str.replace('.mnc', '').str.replace('_.*', '')
        
    dfExpression.index.name = 'Gene'
    
    #Aggregate experiments per gene if flag is set
    if group_experiments:
        print("Aggregating multiple experiments per gene...")
        dfExpression = dfExpression.groupby(dfExpression.index).aggregate(np.mean)
    
    #Remove genes where a threshold of voxels aren't expressing
    fracVoxelsNA = dfExpression.isna().sum(axis=1)/len(dfExpression.columns)
    dfExpression = dfExpression[fracVoxelsNA < missing_threshold]
    
    return dfExpression

True